In [1]:
# from google.colab import drive
# drive.mount('/content/drive')

In [2]:
# !pip install transformers
# !pip install pyspellchecker
# !pip install Keras

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error
from transformers import BertTokenizer
from transformers import TFBertModel
import tensorflow as tf
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score
from spellchecker import SpellChecker
from tqdm import tqdm
from tensorflow.keras.utils import to_categorical
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, cohen_kappa_score
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from prettytable import PrettyTable
import time

import warnings
warnings.filterwarnings("ignore")

In [4]:
# df = pd.read_csv('/content/drive/MyDrive/Preprocessed_df.csv')
# df.head()

In [5]:
def calc_precision(y_true, y_pred, average='macro'):
    """
    Calculates the precision score between the true and predicted values
    """
    precision = precision_score(y_true, y_pred, average=average)
    return precision

def calc_recall(y_true, y_pred, average='macro'):
    """
    Calculates the recall score between the true and predicted values
    """
    recall = recall_score(y_true, y_pred, average=average)
    return recall

def calc_f1_score(y_true, y_pred, average='macro'):
    """
    Calculates the f1-score between the true and predicted values
    """
    f1 = f1_score(y_true, y_pred, average=average)
    return f1

def calc_cohen_kappa_score(y_true, y_pred):
    """
    Calculates the cohen kappa score between the true and predicted values
    """
    kappa_score = cohen_kappa_score(y_true, y_pred, weights = 'quadratic')
    return kappa_score

def calc_accuracy(y_true, y_pred):
    """
    Calculates the accuracy score between the true and predicted values
    """
    accuracy = accuracy_score(y_true, y_pred)
    return accuracy

In [6]:
def print_metrics_function(y_actual, y_predictions):
    
    # Calculate and print accuracy
    accuracy = calc_accuracy(y_actual, y_predictions)
    print("Accuracy:", accuracy)
    
    # Calculate and print precision
    precision = calc_precision(y_actual, y_predictions)
    print("Precision:", precision)

    # Calculate and print recall
    recall = calc_recall(y_actual, y_predictions)
    print("Recall:", recall)

    # Calculate and print f1-score
    f1 = calc_f1_score(y_actual, y_predictions)
    print("F1-Score:", f1)

    # Calculate and print Cohen Kappa Score
    kappa_score = calc_cohen_kappa_score(y_actual, y_predictions)
    print("Cohen Kappa Score:", kappa_score)

    return accuracy, precision, recall, f1, kappa_score

In [7]:
def choose_classifiers(classifier_name = "logistic_regression"):
    """
    Takes a regressor as input and returns a corresponding classifier object
    """
    
    if classifier_name == 'logistic_regression':
        return LogisticRegression()
    elif classifier_name == 'decision_tree_classifier':
        return DecisionTreeClassifier()
    elif classifier_name == 'random_forest_classifier':
        return RandomForestClassifier()
    elif classifier_name == 'gradient_boosting_classifier':
        return GradientBoostingClassifier()
    elif classifier_name == 'adaboost_classifier':
        return AdaBoostClassifier()
    elif classifier_name == 'k_neighbors_classifier':
        return KNeighborsClassifier()
    elif classifier_name == 'support_vector_classifier':
        return SVC()
    elif classifier_name == 'xgboost_classifier':
        return XGBClassifier()
    elif classifier_name == 'gaussian_naive_bayes_classifier':
        return GaussianNB()
    else:
        raise ValueError(f"Classifier {classifier_name} not supported for this problem.")

In [8]:
def dataset_preparation(data, target = 'domain1_score'):
    
    X = data.drop([target], axis = 1)
    y = data[target]
    
    return X, y

In [9]:
def spell_corrector(tokens):
    spell_checker = SpellChecker()
    correct_tokens = []
    for token in tqdm(tokens):
        if spell_checker.correction(token.lower()):
            correct_tokens.append(spell_checker.correction(token.lower()))
        else:
            correct_tokens.append(token.lower())
    
    return ' '.join(correct_tokens)

In [10]:
# Declaring some visualization methods to plot accuracy and model diagram
def plot_accuracy_curve(history):
  import matplotlib.pyplot as plt
  plt.plot(history.history['loss'])
  plt.plot(history.history['mae'])
  plt.title('Model loss')
  plt.ylabel('Loss')
  plt.xlabel('Epoch')
  plt.legend(['Train', 'Test'], loc='upper left')
  plt.show()

def plot_acrchitecture(filename, model):
  from keras.utils import plot_model
  plot_model(model, to_file=str(filename) + '.png')

In [11]:
from keras.layers import Embedding, Input, LSTM, Dense, Dropout, Lambda, Flatten, Bidirectional, Conv2D, Conv1D, MaxPooling1D, GlobalMaxPooling1D
from keras.models import Sequential,Model, load_model, model_from_config
import keras.backend as K

def get_model(Hidden_dim1=400, Hidden_dim2=128, return_sequences = True, dropout=0.2, recurrent_dropout=0.2, input_size=768, activation='relu', bidirectional = False):
    """Define the model."""
    model = Sequential()
    if bidirectional:
        model.add(Bidirectional(LSTM(Hidden_dim1,return_sequences=return_sequences , dropout=0.4, recurrent_dropout=recurrent_dropout), input_shape=[1, input_size]))
        model.add(Bidirectional(LSTM(Hidden_dim2, recurrent_dropout=recurrent_dropout)))
    else:
        model.add(LSTM(Hidden_dim1, dropout=0.4, recurrent_dropout=recurrent_dropout, input_shape=[1, input_size], return_sequences=return_sequences))
        model.add(LSTM(Hidden_dim2, recurrent_dropout=recurrent_dropout))
    model.add(Dropout(dropout))
    model.add(Dense(1, activation=activation))

    model.compile(loss='mean_squared_error', optimizer='rmsprop', metrics=['mae'])
    model.summary()
    return model

def get_model_CNN(Hidden_dim1=400, Hidden_dim2=128, return_sequences = True, dropout=0.5, recurrent_dropout=0.4, input_size=768,output_dims=10380, activation='relu', bidirectional = False):
    """Define the model."""
    inputs = Input(shape=(768,1))
    x = Conv1D(64, 3, strides=1, padding='same', activation='relu')(inputs)
    #Cuts the size of the output in half, maxing over every 2 inputs
    x = MaxPooling1D(pool_size=2)(x)
    x = Conv1D(128, 3, strides=1, padding='same', activation='relu')(x)
    x = GlobalMaxPooling1D()(x) 
    outputs = Dense(output_dims, activation='relu')(x)
    model = Model(inputs=inputs, outputs=outputs, name='CNN')
    model.compile(loss='mean_squared_error', optimizer='adam', metrics=['mae','mse'])
    model.summary()
    return model

In [12]:
def max_list(a, b):

    if len(a) != len(b):
        raise ValueError("Input lists must have the same length")

    max_list = []
    for i in range(len(a)):
        max_list.append(max(a[i], b[i]))

    return max_list


In [13]:
df = pd.read_csv('/content/drive/MyDrive/Preprocessed_df.csv')
df.head()

,essay_id,essay_set,essay,rater1_domain1,rater2_domain1,rater3_domain1,domain1_score,rater1_domain2,rater2_domain2,domain2_score,...,word_len,chars_len,avg_word_length,avg_sentence_length,pos_ratios,num_sentences,num_paragraphs,sentiment_polariy,sentiment_subjectivity,preprocessed_text
0,1,1,"Dear local newspaper, I think effects computer...",4,4,NaN,8,NaN,NaN,NaN,...,386,1875,3.984456,1.0,"{'NNP': 0.031088082901554404, 'JJ': 0.05181347...",16,1,0.310471,0.385613,dear local newspaper think effect computer peo...
1,2,1,"Dear @CAPS1 @CAPS2, I believe that using compu...",5,4,NaN,9,NaN,NaN,NaN,...,464,2288,4.030172,1.0,"{'NNP': 0.03879310344827586, ',': 0.0258620689...",20,1,0.274000,0.613167,dear believe using computer benefit u many way...
2,3,1,"Dear, @CAPS1 @CAPS2 @CAPS3 More and more peopl...",4,3,NaN,7,NaN,NaN,NaN,...,313,1541,4.035144,1.0,"{'NNP': 0.04153354632587859, ',': 0.0287539936...",14,1,0.340393,0.498657,dear people use computer everyone agrees benef...
3,4,1,"Dear Local Newspaper, @CAPS1 I have found that...",5,5,NaN,10,NaN,NaN,NaN,...,611,3165,4.328969,1.0,"{'NNP': 0.11620294599018004, ',': 0.0212765957...",27,1,0.266828,0.441795,dear local newspaper found many expert say com...
4,5,1,"Dear @LOCATION1, I know having computers has a...",4,4,NaN,8,NaN,NaN,NaN,...,517,2569,4.071567,1.0,"{'NNP': 0.017408123791102514, ',': 0.025145067...",30,1,0.199684,0.485814,dear know computer positive effect people comp...


In [14]:
df = df.dropna(axis = 1, how = 'any')
drop_columns = ['essay_id', 'pos_ratios', 'essay', 'rater1_domain1', 'rater2_domain1']
df.drop(drop_columns, axis = 1, inplace = True)


In [15]:
from transformers import TFGPT2Model, GPT2Tokenizer

In [16]:
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
tokenizer.pad_token = tokenizer.eos_token
model = TFGPT2Model.from_pretrained('gpt2')
# df_essay_set = df[df.essay_set == 1]
# X, y = dataset_preparation(df_essay_set)
X_main, y_main = dataset_preparation(df)
# X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle = True, random_state = 101, test_size = 0.2,)
X_main.shape


All model checkpoint layers were used when initializing TFGPT2Model.

All the layers of TFGPT2Model were initialized from the model checkpoint at gpt2.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2Model for predictions without further training.


(12976, 10)

In [17]:
BATCH_SIZE = 16
MAX_LENGTH = 300

train_encodings = tokenizer(list(X_main['preprocessed_text']), truncation=True, padding=True, max_length=MAX_LENGTH, return_tensors='tf')
train_dataset = tf.data.Dataset.from_tensor_slices((dict(train_encodings), y_main)).batch(BATCH_SIZE)

embeddings_train = []
for batch in tqdm(train_dataset):
    embeddings_train.append(model(batch[0]['input_ids'])[0][:, -1, :])
embeddings_train = tf.concat(embeddings_train, axis=0)

100%|██████████| 811/811 [06:21<00:00,  2.12it/s]


In [18]:
embeddings_train.shape

TensorShape([12976, 768])

In [19]:
X_emb = pd.DataFrame(embeddings_train.numpy()).copy()
# y = output_data.copy()

In [22]:
final_lstm=[]
final_bilstm=[]
final_cnn=[]
final_logistic_reg=[]
final_random_forest=[]
final_adaboost=[]
final_k_neighbors=[]
final_svc=[]
for sets in range(1,9):
  X_set = X_emb[df.essay_set == sets]
  y_set= y_main[df.essay_set == sets]
  X=X_main[df.essay_set == sets]
  
  # X, y = dataset_preparation(df_essay_set)

  # X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle = True, random_state = 101, test_size = 0.2,)
  print(" -------------------------------------------------")
  print(" -------------------------------------------------")
  print("\n--------SET {}--------\n".format(sets))
  print(" -------------------------------------------------")
  print(" -------------------------------------------------")
  
  start_time = time.time()

  cv = KFold(n_splits=5, shuffle=True)
  cv_data = cv.split(X_set)
  fold_count =1

  lstm= [-2 for _ in range(5)]
  bilstm= [-2 for _ in range(5)]
  cnn= [-2 for _ in range(5)]
  logistic_reg= [-2 for _ in range(5)]
  random_forest= [-2 for _ in range(5)]
  adaboost= [-2 for _ in range(5)]
  k_neighbors= [-2 for _ in range(5)]
  svc= [-2 for _ in range(5)]


  for traincv, testcv in cv_data:
    print("\n--------Fold {}--------\n".format(fold_count))
    # tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
    # tokenizer.pad_token = tokenizer.eos_token
    # model = TFGPT2Model.from_pretrained('gpt2')
    X_train, X_test, y_train, y_test = X_set.iloc[traincv], X_set.iloc[testcv], y_set.iloc[traincv], y_set.iloc[testcv]
    X_train_temp, X_test_temp=X_main.iloc[traincv],X_main.iloc[testcv]

    embeddings_train=tf.constant(X_train.values)
    embeddings_test=tf.constant(X_test.values)
    
    temp_lstm=[]
    temp_bilstm=[]
    temp_cnn=[]
    temp_logistic_reg=[]
    temp_random_forest=[]
    temp_adaboost=[]
    temp_k_neighbors=[]
    temp_svc=[]

    #LSTM (N,1,L)
    print("-----------------------LSTM-----------------------")

    trainDataVectors=tf.reshape(embeddings_train,[embeddings_train.shape[0],1,embeddings_train.shape[1]])
    testDataVectors=tf.reshape(embeddings_test,[embeddings_test.shape[0],1,embeddings_test.shape[1]]) 
    lstm_model = get_model(bidirectional=False)
    history= lstm_model.fit(trainDataVectors, y_train, batch_size=128, epochs=50)
    y_pred = lstm_model.predict(testDataVectors)
    y_pred = np.around(y_pred)
    np.nan_to_num(y_pred)
    # plot_accuracy_curve(history)
    temp_lstm =list(print_metrics_function(y_test, y_pred))
    lstm= max_list(lstm, temp_lstm)
    
    
    #BiLSTM
    print("-----------------------BiLSTM-----------------------")

    Hidden_dim1=300
    Hidden_dim2=100
    return_sequences = True
    dropout=0.2
    recurrent_dropout=0.2
    input_size=768
    activation='relu'
    bidirectional = True
    batch_size = 64
    epoch = 50
    lstm_model = get_model(Hidden_dim1=Hidden_dim1, Hidden_dim2=Hidden_dim2, return_sequences=return_sequences,
                                dropout=dropout, recurrent_dropout=recurrent_dropout, input_size=input_size,
                                activation=activation, bidirectional=bidirectional)
    history = lstm_model.fit(trainDataVectors, y_train, batch_size=batch_size, epochs=epoch)
    y_pred = lstm_model.predict(testDataVectors)
    y_pred = np.around(y_pred)
    np.nan_to_num(y_pred)
    # plot_accuracy_curve(history)
    temp_bilstm =list(print_metrics_function(y_test, y_pred))
    bilstm= max_list(bilstm, temp_bilstm)
    

    #CNN
    print("-----------------------CNN-----------------------")

    trainDataVectors_=tf.reshape(trainDataVectors,[trainDataVectors.shape[0],trainDataVectors.shape[2],1])
    testDataVectors_=tf.reshape(testDataVectors,[testDataVectors.shape[0],testDataVectors.shape[2],1])  
    cnn_model = get_model_CNN(output_dims=1)
    history=cnn_model.fit(trainDataVectors_, y_train, batch_size=128, epochs=100)
    y_pred = cnn_model.predict(testDataVectors_)
    y_pred = np.around(y_pred)
    np.nan_to_num(y_pred)
    # plot_accuracy_curve(history)
    temp_cnn =list(print_metrics_function(y_test, y_pred))
    cnn= max_list(cnn, temp_cnn)
    
    #New features from feature engineering

    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train_temp.drop(['preprocessed_text'], axis = 1))
    X_test_scaled = scaler.transform(X_test_temp.drop(['preprocessed_text'], axis = 1))
    X_train_features = tf.constant(X_train_scaled.astype('float32'))
    X_test_features = tf.constant(X_test_scaled.astype('float32'))

    embeddings_train = tf.concat([embeddings_train, X_train_features], axis = 1)
    embeddings_test = tf.concat([embeddings_test, X_test_features], axis = 1)
    print()
    print("-----------------------Logistic Regression-----------------------")
    model = choose_classifiers("logistic_regression")
    model.fit(embeddings_train, y_train)
    y_predictions = model.predict(embeddings_test)
    temp_logistic_reg =list(print_metrics_function(y_test, y_predictions))
    logistic_reg= max_list(logistic_reg, temp_logistic_reg)

    print("\n")
    print("-----------------------Random Forest Classifier-----------------------")
    model = choose_classifiers("random_forest_classifier")
    model.fit(embeddings_train, y_train)
    y_predictions = model.predict(embeddings_test)
    temp_random_forest =list(print_metrics_function(y_test, y_predictions))
    random_forest= max_list(random_forest, temp_random_forest)
    
    print("\n")
    print("-----------------------Adaboost Classifier-----------------------")
    model = choose_classifiers("adaboost_classifier")
    model.fit(embeddings_train, y_train)
    y_predictions = model.predict(embeddings_test)
    temp_adaboost =list(print_metrics_function(y_test, y_predictions))
    adaboost= max_list(adaboost, temp_adaboost)
    

    print("\n")
    print("-----------------------K Neibhors Classifier-----------------------")
    model = choose_classifiers("k_neighbors_classifier")
    model.fit(embeddings_train, y_train)
    y_predictions = model.predict(embeddings_test)
    temp_k_neighbors =list(print_metrics_function(y_test, y_predictions))
    k_neighbors= max_list(k_neighbors, temp_k_neighbors)
    

    print("\n")
    print("-----------------------Support Vector Classifier-----------------------")
    model = choose_classifiers("support_vector_classifier")
    model.fit(embeddings_train, y_train)
    y_predictions = model.predict(embeddings_test)
    temp_svc =list(print_metrics_function(y_test, y_predictions))
    svc= max_list(svc, temp_svc)

    fold_count+=1

  end_time = time.time()
  time_taken = end_time - start_time

  print("Time taken: {:.5f} seconds".format(time_taken))

  
  # Define the metric names and their corresponding values
  metric_names = ["Accuracy", "Precision", "Recall", "F1-Score", "Kappa Score"]

  metrics_list=[lstm, bilstm, cnn, logistic_reg, random_forest, adaboost, k_neighbors, svc]
  # Define the ML model names
  model_names = ["LSTM","BiLSTM","CNN","Logistic Regression", "Random Forest Classifier", "Adaboost Classifier", 
                "K Neighbors Classifier", "Support Vector Classifier"]

  # Create a PrettyTable object with the metric names as column headers
  table = PrettyTable(["Model"] + metric_names)

  # Add the metric values for each model as rows to the table
  for i, model_name in enumerate(model_names):
      row = [model_name]
      for j in range(len(metric_names)):
          value = metrics_list[i][j]

          # Format the value with three decimal places
          value_formatted = f"{value:.3f}"
          row.append(value_formatted)

      table.add_row(row)

  # Print the formatted table
  print(table.get_string())
  final_lstm.append(lstm[4])
  final_bilstm.append(bilstm[4])
  final_cnn.append(cnn[4])
  final_logistic_reg.append(logistic_reg[4])
  final_random_forest.append(random_forest[4])
  final_adaboost.append(adaboost[4])
  final_k_neighbors.append(k_neighbors[4])
  final_svc.append(svc[4])




 -------------------------------------------------
 -------------------------------------------------

--------SET 1--------

 -------------------------------------------------
 -------------------------------------------------

--------Fold 1--------

-----------------------LSTM-----------------------


Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_8 (LSTM)               (None, 1, 400)            1870400   
                                                                 
 lstm_9 (LSTM)               (None, 128)               270848    
                                                                 
 dropout_41 (Dropout)        (None, 128)               0         
                                                                 
 dense_5 (Dense)             (None, 1)                 129       
                                                                 
Total params: 2,141,377
Trainable params: 2,141,377
Non-trainable params: 0
_________________________________________________________________
Epoch 1/50
12/12 [==============================] - 8s 18ms/step - loss: 23.5425 - mae: 3.7956
Epoch 2/50
12/12 [==============================] - 0s 17ms/step - loss: 3.0898 -

Accuracy: 0.45938375350140054
Precision: 0.43254285751572574
Recall: 0.4684844622800828
F1-Score: 0.39473295363143257
Cohen Kappa Score: 0.7797129719938694
-----------------------BiLSTM-----------------------


Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bidirectional_4 (Bidirectio  (None, 1, 600)           2565600   
 nal)                                                            
                                                                 
 bidirectional_5 (Bidirectio  (None, 200)              560800    
 nal)                                                            
                                                                 
 dropout_42 (Dropout)        (None, 200)               0         
                                                                 
 dense_6 (Dense)             (None, 1)                 201       
                                                                 
Total params: 3,126,601
Trainable params: 3,126,601
Non-trainable params: 0
_________________________________________________________________
Epoch 1/50
23/23 [==========================

Accuracy: 0.4117647058823529
Precision: 0.09105873550317994
Recall: 0.12776039856331828
F1-Score: 0.08991613746685603
Cohen Kappa Score: 0.22092970169559623

--------Fold 2--------

-----------------------LSTM-----------------------


...

Model: "sequential_81"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bidirectional_80 (Bidirecti  (None, 1, 600)           2565600   
 onal)                                                           
                                                                 
 bidirectional_81 (Bidirecti  (None, 200)              560800    
 onal)                                                           
                                                                 
 dropout_118 (Dropout)       (None, 200)               0         
                                                                 
 dense_120 (Dense)           (None, 1)                 201       
                                                                 
Total params: 3,126,601
Trainable params: 3,126,601
Non-trainable params: 0
_________________________________________________________________
Epoch 1/50
10/10 [=========================

Accuracy: 0.2708333333333333
Precision: 0.010833333333333332
Recall: 0.04
F1-Score: 0.01704918032786885
Cohen Kappa Score: 0.0

--------Fold 5--------

-----------------------LSTM-----------------------


Model: "sequential_82"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_164 (LSTM)             (None, 1, 400)            1870400   
                                                                 
 lstm_165 (LSTM)             (None, 128)               270848    
                                                                 
 dropout_119 (Dropout)       (None, 128)               0         
                                                                 
 dense_122 (Dense)           (None, 1)                 129       
                                                                 
Total params: 2,141,377
Trainable params: 2,141,377
Non-trainable params: 0
_________________________________________________________________
Epoch 1/50
5/5 [==============================] - 6s 36ms/step - loss: 1294.4977 - mae: 35.4906
Epoch 2/50
5/5 [==============================] - 0s 39ms/step - loss: 846.9243

Accuracy: 0.09722222222222222
Precision: 0.0037393162393162395
Recall: 0.038461538461538464
F1-Score: 0.006815968841285297
Cohen Kappa Score: 0.0
-----------------------BiLSTM-----------------------


Model: "sequential_83"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bidirectional_82 (Bidirecti  (None, 1, 600)           2565600   
 onal)                                                           
                                                                 
 bidirectional_83 (Bidirecti  (None, 200)              560800    
 onal)                                                           
                                                                 
 dropout_120 (Dropout)       (None, 200)               0         
                                                                 
 dense_123 (Dense)           (None, 1)                 201       
                                                                 
Total params: 3,126,601
Trainable params: 3,126,601
Non-trainable params: 0
_________________________________________________________________
Epoch 1/50
10/10 [=========================

In [23]:
# Define the metric names and their corresponding values
metric_names = ["Prompt-1","Prompt-2","Prompt-3","Prompt-4","Prompt-5","Prompt-6","Prompt-7","Prompt-8"]

metrics_list=[final_lstm, final_bilstm, final_cnn, final_logistic_reg, final_random_forest, final_adaboost, final_k_neighbors, final_svc]
# Define the ML model names
model_names = ["LSTM","BiLSTM","CNN","Logistic Regression", "Random Forest Classifier", "Adaboost Classifier", 
              "K Neighbors Classifier", "Support Vector Classifier"]

table = PrettyTable(["Model"] + metric_names)

  # Add the metric values for each model as rows to the table
for i, model_name in enumerate(model_names):
    row = [model_name]
    for j in range(len(metric_names)):
        value = metrics_list[i][j]

        # Format the value with three decimal places
        value_formatted = f"{value:.3f}"
        row.append(value_formatted)

    table.add_row(row)

# Print the formatted table
print(table.get_string())


+---------------------------+----------+----------+----------+----------+----------+----------+----------+----------+
|           Model           | Prompt-1 | Prompt-2 | Prompt-3 | Prompt-4 | Prompt-5 | Prompt-6 | Prompt-7 | Prompt-8 |
+---------------------------+----------+----------+----------+----------+----------+----------+----------+----------+
|            LSTM           |  0.780   |  0.610   |  0.690   |  0.655   |  0.753   |  0.600   |  0.590   |  0.000   |
|           BiLSTM          |  0.798   |  0.633   |  0.698   |  0.678   |  0.804   |  0.642   |  0.666   |  0.413   |
|            CNN            |  0.450   |  0.000   |  0.000   |  0.000   |  0.000   |  0.000   |  0.396   |  0.329   |
|    Logistic Regression    |  0.677   |  0.484   |  0.709   |  0.706   |  0.794   |  0.645   |  0.587   |  0.184   |
|  Random Forest Classifier |  0.786   |  0.693   |  0.694   |  0.738   |  0.813   |  0.695   |  0.670   |  0.595   |
|    Adaboost Classifier    |  0.654   |  0.633   |  0.6

In [24]:
final_df = pd.DataFrame(metrics_list)
final_df.to_csv('GPT_2.csv', index=False)

